### 1. 데이터 로딩

In [7]:
import pandas as pd

# # 파일 로딩
# oneword_df = pd.read_excel('../../datasets/oneword_concat.csv.xlsx')
# stopword_df = pd.read_excel('../../datasets/stopword_concat.csv.xlsx')
# replace_df = pd.read_excel('../../datasets/replace_concat.csv.xlsx')

# # CSV로 저장
# oneword_df.to_csv('../../datasets/oneword_concat.csv', index=False)
# stopword_df.to_csv('../../datasets/stopword_concat.csv', index=False)
# replace_df.to_csv('../../datasets/replace_concat.csv', index=False)

In [20]:
# 파일 로딩
db_total = pd.read_csv('../../datasets/DB_Total.csv')
db_stopword = pd.read_csv('../../datasets/stopword_concat.csv')
db_oneword = pd.read_csv('../../datasets/oneword_concat.csv')
db_replace = pd.read_csv('../../datasets/replace_concat.csv')

# 각 파일의 처음 5행을 출력
total_head = db_total.head()
stopword_head = db_stopword.head()
oneword_head = db_oneword.head()
replace_head = db_replace.head()

total_head, stopword_head, oneword_head, replace_head

(   Unnamed: 0                       _id        name  \
 0           0  651277e7ddb81e8e96c6bcbb  Google 사용자   
 1           1  651277e7ddb81e8e96c6bcbc  Google 사용자   
 2           2  651277e7ddb81e8e96c6bcbd  Google 사용자   
 3           3  651277e7ddb81e8e96c6bcbe         한반도   
 4           4  651277e7ddb81e8e96c6bcbf         나수인   
 
                                              reviews                 stars  
 0  입장방식은 변경됐는데 어플은 여전히 바코드만 보여집니다. 짐에서는 더이상 바코드스캔...  별표 5개 만점에 1개를 받았습니다.  
 1  정말 답답하네요 하나. GPS 100프로 신뢰하나요? 시설선택목록에서 안보임 4월말...  별표 5개 만점에 1개를 받았습니다.  
 2  입장하려다 시설검색이 안되서 결국 다른곳에 입장했습니다. 기존에 잘 쓰던 앱이 업데...  별표 5개 만점에 1개를 받았습니다.  
 3  평소 이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...  별표 5개 만점에 5개를 받았습니다.  
 4  아이코젠 덕분에 운동이 더 즐겁고 꾸준하게 하게 되었어요. 사용기록이 꼼꼼히 남고 ...  별표 5개 만점에 5개를 받았습니다.  ,
   stopword
 0        가
 1     가까스로
 2       가량
 3       가령
 4       가민,
   one_char_keyword
 0                값
 1                꿀
 2                꿈
 3                끝
 4                날,

### 2. 데이터 클리닝

In [21]:
# 결측치 검사(760개 검사)
missing_values = db_total.isnull().sum()
missing_values

Unnamed: 0      0
_id             0
name            0
reviews         0
stars         760
dtype: int64

### 텍스트 전처리

In [22]:
# 결측치를 포함하는 행 제거
db_total = db_total.dropna(subset=['stars'])

# 리뷰 텍스트를 소문자로 변환
db_total['reviews'] = db_total['reviews'].str.lower()

# 특수 문자 제거
db_total['reviews'] = db_total['reviews'].str.replace(r'[^a-zA-Z0-9가-힣\s]', ' ', regex=True)

# stopword, oneword, replace 리스트 생성
stopwords = db_stopword['stopword'].tolist()
onewords = db_oneword['one_char_keyword'].tolist()
replace_dict = dict(zip(db_replace['before_replacement'], db_replace['after_replacement']))

# stopword 제거
db_total['reviews'] = db_total['reviews'].apply(lambda x: ' '.join(word for word in x.split() if word not in stopwords))

# oneword 제거
db_total['reviews'] = db_total['reviews'].apply(lambda x: ' '.join(word for word in x.split() if word not in onewords))

# replace 적용
db_total['reviews'] = db_total['reviews'].replace(replace_dict, regex=True)

# 결과 확인
db_total.head()

,Unnamed: 0,_id,name,reviews,stars
0,0,651277e7ddb81e8e96c6bcbb,Google 사용자,좋아요입장방식은 수정됐는데 사이트은 바코드만 운동시설에서는 바코드스캔이 아닌 사이트...,별표 5개 만점에 1개를 받았습니다.
1,1,651277e7ddb81e8e96c6bcbc,Google 사용자,좋아요정말 답답하네요 gps 100프로 신뢰하나요 시설선택목록에서 안보임 4월말 업...,별표 5개 만점에 1개를 받았습니다.
2,2,651277e7ddb81e8e96c6bcbd,Google 사용자,좋아요입장하려다 시설검색이 안되서 다른곳에 입장했습니다 기존에 쓰던 사이트이 개선 ...,별표 5개 만점에 1개를 받았습니다.
3,3,651277e7ddb81e8e96c6bcbe,한반도,좋아요이동이 잦고 여러장소에서 운동하는걸 좋아하는 저에게는 합리적인 아이코젠이 정말...,별표 5개 만점에 5개를 받았습니다.
4,4,651277e7ddb81e8e96c6bcbf,나수인,좋아요아이코젠 덕분에 운동이 즐겁고 꾸준하게 되었어요 사용기록이 꼼꼼히 남고 사용지...,별표 5개 만점에 5개를 받았습니다.


### 4. 레이블링 진행 

In [23]:
# 별점에서 숫자만 정확하게 추출
db_total['numeric_stars'] = db_total['stars'].str.extract(r'(\d)개를 받았습니다').astype(int)

# 별점이 3 이상이면 긍정(1), 그렇지 않으면 부정(0)으로 레이블링
db_total['label'] = db_total['numeric_stars'].apply(lambda x: 1 if x >= 3 else 0)

# 결과 확인 (30개)
db_total[['stars', 'numeric_stars', 'label']].head(30)

,stars,numeric_stars,label
0,별표 5개 만점에 1개를 받았습니다.,1,0
1,별표 5개 만점에 1개를 받았습니다.,1,0
2,별표 5개 만점에 1개를 받았습니다.,1,0
3,별표 5개 만점에 5개를 받았습니다.,5,1
4,별표 5개 만점에 5개를 받았습니다.,5,1
5,별표 5개 만점에 5개를 받았습니다.,5,1
6,별표 5개 만점에 5개를 받았습니다.,5,1
7,별표 5개 만점에 2개를 받았습니다.,2,0
8,별표 5개 만점에 5개를 받았습니다.,5,1
9,별표 5개 만점에 5개를 받았습니다.,5,1


### 5. 피처 엔지니어링

-  1694개의 리뷰 + 5000개의 단어 피처로 변환

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(
    db_total['reviews'], db_total['label'], test_size=0.2, random_state=42)

# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer(max_features=5000)

# 훈련 세트에 대해 TF-IDF 변환 학습
X_train_tfidf = vectorizer.fit_transform(X_train)

# 테스트 세트에 대해 동일한 변환 적용
X_test_tfidf = vectorizer.transform(X_test)

# 변환된 데이터의 크기 확인
X_train_tfidf.shape, X_test_tfidf.shape

((1694, 5000), (424, 5000))

### 6. 모델 훈련 및 평가


- 정확도 (Accuracy): 0.807 (샘플 측정 정확도)
- 정밀도 (Precision): 0.798
- 재현율 (Recall): 0.980
- F1 점수 (F1 Score): 0.880


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 로지스틱 회귀 분류기 객체 생성
clf = LogisticRegression(random_state=42)

# 모델 훈련
clf.fit(X_train_tfidf, y_train)

# 테스트 세트에서 예측
y_pred = clf.predict(X_test_tfidf)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

accuracy, precision, recall, f1

(0.7995283018867925,
 0.7962466487935657,
 0.9705882352941176,
 0.8748159057437408)

### 7. 긍정 및 부정 예측:


- predicted_label : (긍정: 1, 부정: 0)
- condition_positive (긍정 True , 부정 False)
- condition_negative (부정 True, 아닌경우 False)

In [34]:
# X_test와 y_pred를 DataFrame으로 결합
predicted_df = pd.DataFrame({'review': X_test, 'predicted_label': y_pred})

# 긍정과 부정의 조건 설정
predicted_df['condition_positive'] = predicted_df['predicted_label'] == 1
predicted_df['condition_negative'] = predicted_df['predicted_label'] == 0

# 결과 확인
predicted_df.head()

,review,predicted_label,condition_positive,condition_negative
1565,좋아요좋아요좋아요좋아요좋아요운동시설사용 간편해요 최고에요,1,True,False
1100,좋아요좋아요좋아요좋아요좋아요다른 운동시설과 다르게 운동시설박스 사이트 운영으로 입출...,1,True,False
1329,좋아요좋아요좋아요좋아요좋아요사이트 덕분에 운동시설 사용이 간편하고 운동하는 분위기도...,1,True,False
1261,좋아요좋아요좋아요좋아요좋아요시설도 좋고 다른데는 짧게 다녔는데 여기는 오래 다닐 있...,1,True,False
637,좋아요좋아요좋아요좋아요좋아요구버젼은 바코드 표시가 전부였는데 사용내역도 확인되고 개...,1,True,False


In [26]:
## 대체어만 따로 출력 

In [25]:
# # before_replacement와 after_replacement를 딕셔너리로 변환
# replace_dict = dict(zip(replace['before_replacement'], replace['after_replacement']))

# # reviews 컬럼의 텍스트를 교체
# db_total['reviews'] = db_total['reviews'].replace(replace_dict, regex=True)

# # 결과 확인
# db_total['reviews'].head()